In [3]:
import pandas as pd
import os
import numpy as np
import time
import random
import timeit
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.autograd import Variable
from copy import deepcopy

In [61]:
def filterByValue (sequence, value):
    for thing in sequence:
        if value in thing:
            yield thing

tracknames = os.listdir("train_data/")
tracknames = filterByValue(tracknames, "f-speedway")
data = []

for i, file in  enumerate(tracknames):
    path = os.path.join('train_data', file)
    print(path)
    rows = pd.read_csv(path,sep=';', index_col=False)
    data.append(rows)

rawtrain = pd.concat(data)

rawtrain.head()

train_data/oval_f-speedway_1cars_2910370.csv
train_data/oval_f-speedway_1cars_29103653.csv
train_data/oval_f-speedway_1cars_2901726.csv
train_data/oval_f-speedway_1cars_29103628.csv
train_data/oval_f-speedway_1cars_29103648.csv
train_data/oval_f-speedway_1cars_2901739.csv
train_data/oval_f-speedway_1cars_2910365.csv
train_data/oval_f-speedway_1cars_2910361.csv
train_data/oval_f-speedway_1cars_29103635.csv
train_data/oval_f-speedway_1cars_2901718.csv
train_data/oval_f-speedway_1cars_29103616.csv
train_data/oval_f-speedway_1cars_29103632.csv
train_data/oval_f-speedway_1cars_2901755.csv
train_data/oval_f-speedway_1cars_29103640.csv
train_data/oval_f-speedway_1cars_29103620.csv
train_data/oval_f-speedway_1cars_29103630.csv
train_data/oval_f-speedway_1cars_29103625.csv
train_data/oval_f-speedway_1cars_29103645.csv
train_data/oval_f-speedway_1cars_29103637.csv
train_data/oval_f-speedway_1cars_2901752.csv
train_data/oval_f-speedway_1cars_2901742.csv
train_data/oval_f-speedway_1cars_29103618.c

,accel,brake,steer,angle,curLapTime,distFromStart,distRaced,gear,lastLapTime,racePos,...,oppos26,oppos27,oppos28,oppos29,oppos30,oppos31,oppos32,oppos33,oppos34,oppos35
0,1.0,0,0.0,0.016239,-0.982,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
1,1.0,0,0.0,0.016239,-0.962,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
2,1.0,0,0.0,0.016239,-0.942,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
3,1.0,0,0.0,0.016239,-0.922,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200
4,1.0,0,0.0,0.016239,-0.902,3678.83,0.0,0,0.0,1,...,200,200,200,200,200,200,200,200,200,200


In [62]:
# preprocess data
train = deepcopy(rawtrain)

# merge accel and brake into accelbrake
def mergeAccelBrake (row):
    brake = row['brake']
    if brake > 0.0:
        return -1
    else:
        return row['accel']

if not 'accelbrake' in train.columns:
    train.insert(0, 'accelbrake', 0.0, False)

#startTime = timeit.default_timer()
#train['accelbrake'] = train.apply(lambda row: mergeAccelBrake(row), axis=1)
#train = train.apply(lambda row: normalizeSomeColumns(row), axis=1)
train['steer'] /= 2.0
train['steer'] += 0.5
train['speedX'] /= 200.0
train.iloc[:, 21:] /= 200.0

#elapsedTime = timeit.default_timer() - startTime
#print("preprocessing took {:.1f}s".format(elapsedTime))
train.iloc[150:160, :]

,accelbrake,accel,brake,steer,angle,curLapTime,distFromStart,distRaced,gear,lastLapTime,...,oppos26,oppos27,oppos28,oppos29,oppos30,oppos31,oppos32,oppos33,oppos34,oppos35
150,0.0,1.0,0,0.543868,0.572271,2.050,3696.84,18.0049,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
151,0.0,1.0,0,0.543983,0.560527,2.072,3697.21,18.3799,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
152,0.0,1.0,0,0.543646,0.548705,2.094,3697.60,18.7610,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
153,0.0,1.0,0,0.544537,0.537002,2.116,3697.98,19.1484,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
154,0.0,1.0,0,0.545665,0.525451,2.138,3698.38,19.5425,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
155,0.0,1.0,0,0.546106,0.513925,2.160,3698.78,19.9426,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
156,0.0,1.0,0,0.549268,0.502583,2.182,3699.18,20.3494,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
157,0.0,1.0,0,0.551185,0.491511,2.204,3699.60,20.7617,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
158,0.0,1.0,0,0.554310,0.480633,2.226,3700.01,21.1799,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
159,0.0,1.0,0,0.556201,0.469859,2.248,3700.44,21.6040,1,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [63]:

inputColumns = list(range(4,5)) + list(range(12,17)) + list(range(21,len(train.columns)))
inp = train.iloc[:,inputColumns]
outp = train.iloc[:,1:4]
print(outp.iloc[0])
print("input columns: {}".format(inp.columns))
print("output columns: {}".format(outp.columns))
input_size = inp.shape[1]
output_size = outp.shape[1]
datapoints = inp.shape[0]
print(datapoints,input_size,output_size)

accel    1.0
brake    0.0
steer    0.5
Name: 0, dtype: float64
input columns: Index(['angle', 'speedX', 'speedY', 'speedZ', 'trackPos', 'z', 'track00',
       'track01', 'track02', 'track03', 'track04', 'track05', 'track06',
       'track07', 'track08', 'track09', 'track10', 'track11', 'track12',
       'track13', 'track14', 'track15', 'track16', 'track17', 'track18',
       'oppos00', 'oppos01', 'oppos02', 'oppos03', 'oppos04', 'oppos05',
       'oppos06', 'oppos07', 'oppos08', 'oppos09', 'oppos10', 'oppos11',
       'oppos12', 'oppos13', 'oppos14', 'oppos15', 'oppos16', 'oppos17',
       'oppos18', 'oppos19', 'oppos20', 'oppos21', 'oppos22', 'oppos23',
       'oppos24', 'oppos25', 'oppos26', 'oppos27', 'oppos28', 'oppos29',
       'oppos30', 'oppos31', 'oppos32', 'oppos33', 'oppos34', 'oppos35'],
      dtype='object')
output columns: Index(['accel', 'brake', 'steer'], dtype='object')
412957 61 3


In [84]:
# Hyper Parameters
input_size = inp.shape[1]
hidden_size = 50
output_size = outp.shape[1]
num_epochs = 5
batch_size = 100

In [65]:

# Neural Network Model
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h = F.sigmoid(self.fc1(x))
        h = F.sigmoid(self.fc2(h))
        h = F.sigmoid(self.fc3(h))
        return h

In [7]:
# def minibatch(data, batch_size=32):
#     for i in range(0, len(data), batch_size):
#         yield data[i:i+batch_size]

In [67]:
def get_variable(data, labels = False):
    """Get a Variable given data (pandas dataframe)"""
    # Use float (not double) for better precission: (see reply of smth)
    # https://discuss.pytorch.org/t/problems-with-target-arrays-of-int-int32-types-in-loss-functions/140/3
    
    # Labels need to be long to work (for cross entropy, for MSE they both need to be long)
    if labels:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
    else:
        tensor = torch.from_numpy(data.iloc[:,:].as_matrix()).float()
#     print(tensor)
    return Variable(tensor)

In [72]:
def preprocess(batch):
    """ ... """
    
    inputColumns = list(range(4,5)) + list(range(12,17)) + list(range(21,len(train.columns)))
    
    x = batch.iloc[:,inputColumns]
    y = batch.iloc[:,1:4]
    
    return x, y

In [69]:
model = Net(input_size, hidden_size, output_size)

In [178]:
#TO CUDA
model.cuda()

Net (
  (fc1): Linear (61 -> 50)
  (fc2): Linear (50 -> 50)
  (fc3): Linear (50 -> 3)
)

In [70]:
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [96]:
for ITER in range(num_epochs):
    
    # Take a random sample of size batchsize
    
    train_loss = 0.0
    start = time.time()
    updates = 0

    for i, batch in train.iterrows():
#         batch = train.iloc[i,:]
        print(batch.datatype())
        updates += 1

        # pad data with zeros
        x, y = preprocess(batch)
        #print(y)
        # forward pass
        scores = model(get_variable(x))
        #print(scores)
        targets = get_variable(y, labels= True)
        #print(targets)
        loss = nn.MSELoss(reduce = False)
        #loss = scores - targets
        output = loss(scores, targets)
        #print(output)
        #train_loss += np.mean(loss.data)

        # backward pass
        model.zero_grad()
        loss.backward()

        # update weights
        optimizer.step()
        break

    print("iter %r: avg train loss=%.4f, time=%.2fs" %
          (ITER, train_loss/updates, time.time()-start))
    #print(testPrediction())
    break

    # evaluate
#     _, _, acc_train = evaluate(model, train)
#     _, _, acc_dev = evaluate(model, dev)
#     print("iter %r: train acc=%.4f  test acc=%.4f" % (ITER, acc_train, acc_dev))

AttributeError: 'Series' object has no attribute 'datatype'

In [75]:
save_path_model = 'simple_ff_model_30epochs_batch100_dict_hiden_size50_2HL_INP72_outp3_Forward_sig3_onlyrelevantdata.pt'
save_path_dict = 'simple_ff_DICT_30epochs_batch100_dict_hiden_size50_2HL_INP72_outp3_Forward_sig3_onlyrelevantdata.pt'
# https://stackoverflow.com/questions/42703500/best-way-to-save-a-trained-model-in-pytorch
torch.save(model, save_path_model)
torch.save(model.state_dict(),save_path_dict)
# model.save_state_dict('trainedmodel

/home/jasper-ubuntu/miniconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [18]:
path = os.path.join('','simple_ff_DICT_30epochs_batch100_dict_hiden_size50_2HL_INP72_outp3_Forward_sig3_onlyrelevantdata.pt')

model = Net(input_size, hidden_size, output_size)
model.load_state_dict(torch.load(path))